# BSDE in high dimension : example of a European Option

In [1]:

import numpy as np
import scipy as sp
import pylab as pl
import matplotlib.pyplot as plt
import statistics as st
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
import pandas as pd


In [2]:
                                        "In case we want to add some randomness in Stocks"

def drifts(p): 
    drifts_vector=np.zeros([p,1])
    for  i in range(p): 
        drifts_vector[i]=np.random.uniform(0.1,0.5)
    return drifts_vector

In [3]:
                                        "For the case we want correlation"

def corr_matrix(p): 
    rho=np.zeros([p,p])
    for i in range (p): 
        for j in range (p) :
            if (i==j):
                rho[i,j]=1
            else: 
                rho[i,j]=np.random.uniform(-1,1)
    return rho

In [4]:
def generate_paths(S0, dt , sigma, r, N, m,q):
    '''
    Parameters
    ==========
    S0    : float
            positive, initial Stock Value
    dt    : float
            time step discretization
    sigma : float 
            volatility 
    r     : float
            bond return 
    N     : int 
            Number of paths generated      
    m     : int 
            number of steps
    
    Returns
    =======
    S : matrix
        generated paths
    '''
    S = np.zeros((m + 1, N))
    S[0] = S0
    for t in range(1, m + 1):
        rand = np.random.standard_normal(N)
        S[t] = S[t - 1] * np.exp((r -q- sigma ** 2 / 2) * dt + sigma * dt ** 0.5 * rand)
    return S

In [5]:
def generate_multiple_asset(p, S0, dt , sigma, r, N, m,q):
    S = []
    
    #Generate p assets
    for __ in range (N): 
        S_asset=generate_paths(S0, dt , sigma, r, p, m,q)
        S.append(S_asset)
    return (S)

In [6]:
def exercise_matrix(S,p,N,K,m): 
    payoff=np.zeros((m + 1, N))
    for j in range (N): 
        for i in range (m + 1):
            pay_int = np.prod(S[j][i,:])**(1/p)
            pay=max(pay_int-K,0)
            payoff[i,j]=pay
    return (payoff)

In [8]:
                                            "Pricing of a basket option"



def pricing_option(K,r,sigma,p,m,S_0,T, RF_n_estimators,RF_max_leaf_nodes,N,q):
  
    #Time-step
    dt=T/m
    
    #Discount factor
    df=1/(1+r*dt)
    #N simulations of p underlying assets
    S = generate_multiple_asset(p, S_0, dt , sigma, r, N, m,q)
    #Matrix of exercise prices
    h = exercise_matrix(S,p,N,K,m)   
    #price of the option at time T = Initialization for a basket option
    Y = h[-1]
     
    #Iteration over time
    for t in range (m-1,0,-1): 
        
        #Creation of the matrix N*p to regress
        X=np.zeros([N,p])
        for i in range (N):
            for j in range (p): 
                X[i,j]=S[i][t , j - 1]
    
        rf=RandomForestRegressor(n_estimators=RF_n_estimators,
                                max_leaf_nodes=RF_max_leaf_nodes,oob_score=False, n_jobs=-1)
        rf.fit(X,Y)
        Y=rf.predict(X)*df
     
    Y_opt= df * sum(Y)/N
    return (Y_opt)

In [9]:
"""Pricing with BS formula"""

def black_scholes(K,r,sigma, S_0,T,p,q): 
    d=sigma**2*(1-1/p)/2+q
    d1=1/(sigma*np.sqrt(T/p))*(np.log(S_0/K)+(r+0.5*sigma*sigma/p-d)*T)
    d2=d1-sigma*np.sqrt(T/p)
    dscnt=np.exp(-r*T)
    call=-norm.cdf(d2)*K*dscnt+norm.cdf(d1)*S_0*np.exp(-d*T)
    return (call)

In [11]:

T=1
m=12
K=100
S_0=100
r=0.06
sigma=0.4
p=7
RF_n_estimators = 10
RF_max_leaf_nodes = 5
N=10000
q=0.05


"We price"    
price_BSDE=pricing_option(K,r,sigma,p,m,S_0,T, RF_n_estimators,RF_max_leaf_nodes,N,q)
print("price",price_BSDE)

price_BS=black_scholes(K,r,sigma, S_0,T,p,q)
print ("price Black-Scholes ", price_BS) 

price 3.37610715601
price Black-Scholes  3.24229409757
